In [1]:
### Testing for updateTrashCanStatus
from CompareImagesModel import *

model = CompareImagesModel()

c:\Users\krome\OneDrive\Desktop\innovaite\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\krome\OneDrive\Desktop\innovaite\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
import numpy as np
import pandas as pd
import os

def load_image_paths_from_folders_to_df(base_path):
    '''
    Provide base path for images and will return a pandas dataframe with the image paths and labels.
    '''
    image_paths = []
    labels = []
    label_names = os.listdir(base_path)

    for label_idx, label_name in enumerate(label_names):
        label_folder = os.path.join(base_path, label_name)

        if os.path.isdir(label_folder):
            for image_name in os.listdir(label_folder):
                image_path = os.path.join(label_folder, image_name)

                if image_path.endswith(('.png', '.jpg', '.jpeg')):  # Filter by image extensions
                    image_paths.append(image_path)
                    labels.append(label_name)  # Label is the name of the folder in label_names

    df = pd.DataFrame({'IMAGEPATH': image_paths, 'ID': labels })

    return df

In [3]:
Cleaned = "../../Trash Cans Cleaned"  # Replace with your destination path
trashcan_images_df = load_image_paths_from_folders_to_df(Cleaned)
trashcan_images_df['ID'] = trashcan_images_df['ID'].astype(int)

In [4]:
trashcan_images_df["ID"].unique()

array([ 1, 16, 17, 18, 19,  2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,  3,
       30, 31, 32, 33, 38, 39,  4,  5,  6,  7])

In [5]:
from PIL import Image
# Load the baseline image and extract its features
trashcan_id = 21
# get all image paths for trashcan of trashcan_id
filtered_paths = trashcan_images_df[trashcan_images_df['ID'] == trashcan_id]["IMAGEPATH"].values
print(type(filtered_paths))

baseline_img = Image.open(filtered_paths[0])
other_img = Image.open(filtered_paths[1])

print(model.predict(baseline_img, other_img))

<class 'numpy.ndarray'>
0.84914136


In [ ]:
from updateTrashCanStatus import matchTrashCanID
trash_can_ids = [20] #[19, 20, 21, 22]
image_folder_paths = []
for id in trash_can_ids:
    paths = trashcan_images_df[trashcan_images_df['ID'] == id]["IMAGEPATH"].values
    image_folder_paths.append(os.path.dirname(paths[0]))

trashcan_id = 20
filtered_paths = trashcan_images_df[trashcan_images_df['ID'] == trashcan_id]["IMAGEPATH"].values
image = Image.open(filtered_paths[0])

# pred_id = matchTrashCanID(image, image_folder_paths, trash_can_ids, model)

# print(trash_can_ids)
# print(image_folder_paths)
# print("Expected:", trashcan_id, "Returned:", pred_id)

[20]
['../../Trash Cans Cleaned\\20']
Expected: 20 Returned: 20


In [7]:
import updateTrashCanStatus
import importlib
importlib.reload(updateTrashCanStatus)

latitude, longitude = 42.3393828, -71.0871835 #42, -71
updateTrashCanStatus.updateTrashCanStatusMain(image=image, latitude=latitude, longitude=longitude,full_likelihood=1, model=model)

min lat: 42.338382800000005 max_lat: 42.3403828
min long: -71.0881835 max long: -71.08618349999999
image_folder_paths: ['../Trash Cans Cleaned/1', '../Trash Cans Cleaned/2', '../Trash Cans Cleaned/3', '../Trash Cans Cleaned/16', '../Trash Cans Cleaned/17', '../Trash Cans Cleaned/18', '../Trash Cans Cleaned/20', '../Trash Cans Cleaned/38', '../Trash Cans Cleaned/39']
IDs: [1, 2, 3, 16, 17, 18, 20, 38, 39]
ID: 20
Marking trash can 20 as FULL.


In [14]:
import sqlite3

# Create (or connect to) the database
conn = sqlite3.connect("database.db")
cursor = conn.cursor()

# Read the .sql file
with open("trash_can.sql", "r") as f:
    sql_script = f.read()

# Execute the SQL script
cursor.executescript(sql_script)

# Commit and close
conn.commit()
conn.close()


In [5]:
from updateTrashCanStatus import updateTrashCanStatus
DATABASE_PATH = "database.db"
import sqlite3

# Call function after defining config
trash_can_id = 19
updateTrashCanStatus(trash_can_id, 1)

# Connect to database and verify result
try:
    connection = sqlite3.connect(DATABASE_PATH)
    cursor = connection.cursor()

    # Query the updated row
    cursor.execute(f"SELECT * FROM trash_cans WHERE trash_can_id = {trash_can_id};")
    result = cursor.fetchone()

    print("\nDatabase Output:")
    print(result)  # This should print the updated trash can row

except Exception as e:
    print(f"Database Error: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Marking trash can 19 as FULL.

Database Output:
(19, 42.338979, -71.0885834, 1, '2025-02-16 07:21:28')


In [15]:
from updateTrashCanStatus import getNearbyTrashCanImages
latitude, longitude = 42.3393828, -71.0871835 #42, -71
image_folder_paths, trash_can_ids = getNearbyTrashCanImages(latitude, longitude, range=0.001)
print(trash_can_ids)
print(image_folder_paths)

min lat: 42.338382800000005 max_lat: 42.3403828
min long: -71.0881835 max long: -71.08618349999999
[1, 2, 3, 16, 17, 18, 20, 38, 39]
[['Trash Cans Cleaned/1/'], ['Trash Cans Cleaned/2/'], ['Trash Cans Cleaned/3/'], ['Trash Cans Cleaned/16/'], ['Trash Cans Cleaned/17/'], ['Trash Cans Cleaned/18/'], ['Trash Cans Cleaned/20/'], ['Trash Cans Cleaned/38/'], ['Trash Cans Cleaned/39/']]


In [3]:
from FullnessModel import *

In [4]:
fullness_model = FullnessModel()

TypeError: FullnessModel.__init__() missing 2 required positional arguments: 'train_dir' and 'validate_dir'

In [2]:
! pip install tensorflow

  Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl.metadata (4.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.17.2-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
  Using cached grpcio-1.70.0-cp311-cp311-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Usin

  You can safely remove it manually.
  You can safely remove it manually.
